In [1]:
import os

import dask.dataframe as ddf
import numpy as np
import pandas as pd

from dask.distributed import LocalCluster, Client

# Dask

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
file_lst = [file.path for file in os.scandir('dossier/') if file.is_file() and file.path.endswith('.csv')]

In [4]:
file_lst

['dossier/books_batch_001-100.csv',
 'dossier/books_batch_101-200.csv',
 'dossier/books_batch_201-300.csv',
 'dossier/books_batch_301-400.csv',
 'dossier/books_batch_401-500.csv',
 'dossier/books_batch_501-543.csv']

In [5]:
def read_files(files : list, **kwargs) -> ddf:
    """Read in multiple files from a list and return a lazy dask dataframe"""
    return ddf.read_csv(files, **kwargs)

In [6]:
delayed_df = read_files(files=file_lst, 
                        usecols=["isbn", "isbn13", "title", "authors","language", "publisher"], 
                        dtype={"isbn": "object"}
                       )

In [7]:
books = delayed_df.compute()

In [8]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271197 entries, 0 to 21345
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   publisher  270950 non-null  object
 1   language   271190 non-null  object
 2   authors    271175 non-null  object
 3   title      271195 non-null  object
 4   isbn13     271197 non-null  int64 
 5   isbn       271197 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.5+ MB


In [9]:
books.head()

,publisher,language,authors,title,isbn13,isbn
0,Viking Children's Books,en,"['Westall, Robert']",Urn Burial,9780670815371,0670815373
1,Distribooks,de,"['Coelho, Paulo']",Veronika beschliesst zu sterben: Roman,9783257233056,3257233051
2,Ballantine Books,en,"['Adams, Douglas', 'Carwardine, Mark']",Last Chance to See,9780345371980,0345371984
3,Ace,en,"['Heinlein, Robert A.']",Starship Troopers,9780441783588,0441783589
4,Crown,en,"['Anderson, Joan']",A Year By The Sea: Thoughts of an Unfinished W...,9780767905930,0767905938


In [10]:
books.isnull().sum()

publisher    247
language       7
authors       22
title          2
isbn13         0
isbn           0
dtype: int64

In [11]:
books = books.dropna(subset=["publisher", "authors", "language", "title"])

In [12]:
books.isnull().sum()

publisher    0
language     0
authors      0
title        0
isbn13       0
isbn         0
dtype: int64

In [13]:
ratings = ddf.read_csv("archive/Ratings.csv").compute()

In [14]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [15]:
ratings.columns = ratings.columns.str.replace('-','_').str.lower()

In [16]:
ratings.head()

,user_id,isbn,book_rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


#### Most Popular items

In [17]:
books["title"].value_counts()

title
The Secret Garden                                                                          22
Little Women                                                                               19
The Night Before Christmas                                                                 18
Dracula                                                                                    18
The Hobbit                                                                                 17
                                                                                           ..
A Gentle Giving                                                                             1
Guide to the Recommended Country Inns of the West Coast: California, Oregon, Washington     1
Game Without Rules                                                                          1
Accidental Journey: A Cambridge intern's memory of World War II                             1
The Perfect Gift (Two of a Kind, 26)                  

In [18]:
explicit_rating = ratings.query("book_rating != 0")

In [19]:
explicit_rating.head()

,user_id,isbn,book_rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6


#### Merge 
Merge the `ratings` and `books` dataframes and rearrange the columns

In [20]:
ratings_books = explicit_rating.merge(books, on=["isbn"])

In [21]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388670 entries, 0 to 388669
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      388670 non-null  int64 
 1   isbn         388670 non-null  object
 2   book_rating  388670 non-null  int64 
 3   publisher    388670 non-null  object
 4   language     388670 non-null  object
 5   authors      388670 non-null  object
 6   title        388670 non-null  object
 7   isbn13       388670 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 23.7+ MB


In [22]:
ratings_books = ratings_books[["user_id", "isbn13", "title", "authors", "publisher", "language", "book_rating"]]
ratings_books.head()

,user_id,isbn13,title,authors,publisher,language,book_rating
0,276726,9780155061224,Rites of Passage,"['Rae, Judith']",Wadsworth Publishing,en,5
1,276729,9780521656153,Help! Level 1 (Cambridge English Readers),"['Prowse, Philip']",Cambridge University Press,en,3
2,276729,9780521795029,The Amsterdam Connection Level 4 (Cambridge En...,"['Leather, Sue']",Cambridge University Press,en,6
3,276744,9780385501200,A Painted House: A Novel,"['Grisham, John']",Doubleday,en,7
4,11676,9780385501200,A Painted House: A Novel,"['Grisham, John']",Doubleday,en,10


#### Most liked items

In [23]:
avg_rating = ratings_books[["title", "book_rating"]].groupby(["title"]).mean().sort_values(by="book_rating", ascending=False)
avg_rating.reset_index()

,title,book_rating
0,"Molly's Surprise: A Christmas Story, Book Thre...",10.0
1,The Art Of Playing Mythos The Cthulhu Collecta...,10.0
2,The Archaic Revival: Speculations on Psychedel...,10.0
3,The Archer King,10.0
4,The Architects of Hyperspace,10.0
...,...,...
135999,The New Beverly Hills Diet: The latest weight-...,1.0
136000,Let's Go 2000: Europe: The World's Bestselling...,1.0
136001,Rencontre au bord du fleuve,1.0
136002,Honey for a Child's Heart: The Imaginative Use...,1.0


#### Note:

Some entries may, due to their paucity, skew the results. Reviews on either the high or low end may come from a limited number of reviews. Case in point: one of the lowest rated books, `I Hate the Dallas Cowboys: And Who Elected Them America's Team Anyway?` has a score of 1.0 from a single review. The same is also true of one of the highest rated books, `Molly's Surprise: A Christmas Story, Book Three (The American Girls Collection)`

Since such books are not representative of general reader sentiment, it is better to get rid of any books with fewer than 10 reviews in order to establish a truer baseline sentiment.

In [24]:
(ratings_books["title"] == "I Hate the Dallas Cowboys: And Who Elected Them America's Team Anyway?").sum()

1

In [25]:
(ratings_books["title"] == "Molly's Surprise: A Christmas Story, Book Three (The American Girls Collection)").sum()

1

#### Filter out books with fewer than 10 reviews:

In [26]:
frequency = ratings_books["title"].value_counts()

In [27]:
frequently_reviewed = frequency[frequency > 10].index

In [28]:
books_df = ratings_books[ratings_books["title"].isin(frequently_reviewed)]

In [29]:
books_df.head()

,user_id,isbn13,title,authors,publisher,language,book_rating
3,276744,9780385501200,A Painted House: A Novel,"['Grisham, John']",Doubleday,en,7
4,11676,9780385501200,A Painted House: A Novel,"['Grisham, John']",Doubleday,en,10
5,16877,9780385501200,A Painted House: A Novel,"['Grisham, John']",Doubleday,en,9
6,17975,9780385501200,A Painted House: A Novel,"['Grisham, John']",Doubleday,en,6
7,20806,9780385501200,A Painted House: A Novel,"['Grisham, John']",Doubleday,en,6


In [30]:
books_df["title"].value_counts()

title
The Lovely Bones                                                    754
Harry Potter and the Sorcerer's Stone                               629
Wild Animus: A Novel                                                581
A Painted House                                                     562
Snow Falling on Cedars                                              532
                                                                   ... 
Engaged to Die (Death on Demand Mysteries, No. 14)                   11
The Woman Who Walked into Doors: A Novel (A Paula Spencer Novel)     11
Nerve                                                                11
King of the Wind: The Story of the Godolphin Arabian                 11
Spellbound                                                           11
Name: count, Length: 5128, dtype: int64

#### Filter out less common languages

The same process can be repeated for lanuages. There are only two books written in Russian and one in Hindi. These can be removed since no model can make reasonable predictions based on so few values. The eight books written in `en_US` will be merged into the `en` language category 

In [31]:
books_df.loc[books_df["language"] == "en_US", "language"] = "en"

In [32]:
books_df["language"].value_counts()

language
en    154864
de      1223
es       361
fr       262
it       195
ru         2
hi         1
Name: count, dtype: int64

In [33]:
minor_langs = books_df.loc[(books_df["language"] == "ru") | (books_df["language"] == "hi")].index

In [34]:
books_df = books_df.drop(minor_langs, axis=0)

In [35]:
books_df["language"].value_counts()

language
en    154864
de      1223
es       361
fr       262
it       195
Name: count, dtype: int64

#### Filter out users with few ratings

A user should have at rated at least 5 books for their recommendations to be included


In [36]:
usrs_freq = books_df["user_id"].value_counts()

In [37]:
prolific_readers = usrs_freq[usrs_freq > 5].index

In [38]:
books_df = books_df[books_df["user_id"].isin(prolific_readers)]

### Average Rating of Remaining Books

In [39]:
avg_popular = books_df[["title", "book_rating"]].groupby("title").mean().reset_index().sort_values(by="book_rating", ascending=False)
avg_popular

,title,book_rating
2778,Postmarked Yesteryear: 30 Rare Holiday Postcards,10.000000
2127,Liebesleben (German Edition),10.000000
4412,"The Secret Daughter: Raising Cane, Book 2 (Har...",10.000000
2835,Purple Cow: Transform Your Business by Being R...,10.000000
1088,Dilbert: A Book of Postcards,9.923077
...,...,...
3671,The Coldest Winter Ever,4.666667
5019,Wild Animus: A Novel,4.036364
845,Confessions of a Sociopathic Social Climber: T...,3.923077
2089,Le Crime de L'Orient-Express (Le Livre de Poch...,3.000000


In [40]:
grouped = books_df.groupby("title")[["book_rating"]].agg({'book_rating' : [('num_ratings', 'count')]})
grouped.columns = grouped.columns.droplevel()
grouped = grouped.reset_index().sort_values(by="num_ratings", ascending=False)
grouped

,title,num_ratings
1649,Harry Potter and the Sorcerer's Stone,390
4122,The Lovely Bones,372
1647,Harry Potter and the Order of the Phoenix (Boo...,321
134,A Painted House,314
3843,The Firm,299
...,...,...
2556,Novecento: Un Monologo (Universale Economica F...,1
1464,Free,1
1473,Friedhof der Kuscheltiere. Roman.,1
4695,Theos Reise. Roman über die Religionen der Welt.,1


The highest rated book, `Postmarked Yesteryear: 30 Rare Holiday Postcards` has a score of 10.0 with 11 ratings. 

In [41]:
grouped[grouped["title"] == "Postmarked Yesteryear: 30 Rare Holiday Postcards" ]

,title,num_ratings
2778,Postmarked Yesteryear: 30 Rare Holiday Postcards,9


---
### Non-Personalized Recommendations

These are recommendations made to all users without taking their preferences into account. One example is to recommend items most commonly seen together. 

To accomplish this, record every time two books were read by the same person, and then count how often these pairings of books occur. Use the resulting lookup table to suggest books that are often read by the same people, implying that if they like one, they're likely to enjoy the other.

In [42]:
from itertools import permutations

In [43]:
def create_pairs(col):
    """Return pairs of books that are frequently read together"""
    pairs = pd.DataFrame(list(permutations(col.values, 2)), columns=["book_a", "book_b"])
    return pairs

In [44]:
book_pairs = books_df.groupby("user_id")["title"].apply(create_pairs)

In [45]:
book_pairs = book_pairs.reset_index(drop=True)

The following pairs of books are commonly read together

In [46]:
book_pairs.head()

,book_a,book_b
0,Angels & Demons,The Beach House
1,Angels & Demons,The Alibi
2,Angels & Demons,A Kiss Remembered
3,Angels & Demons,The Short Forever (A Stone Barrington Novel)
4,Angels & Demons,Dead Aim


In [47]:
pair_counts = book_pairs.groupby(["book_a", "book_b"]).size()

In [48]:
pair_counts_df = pair_counts.to_frame(name="size").reset_index().sort_values(by="size", ascending=False)

In [49]:
true_pairs = pair_counts_df[pair_counts_df["book_a"] != pair_counts_df["book_b"]]

In [50]:
true_pairs.head()

,book_a,book_b,size
2025844,Harry Potter and the Order of the Phoenix (Boo...,Harry Potter And The Goblet Of Fire,175
2010197,Harry Potter And The Goblet Of Fire,Harry Potter and the Order of the Phoenix (Boo...,175
2025848,Harry Potter and the Order of the Phoenix (Boo...,Harry Potter The Illustrated 4 Books Collectio...,158
2019096,Harry Potter The Illustrated 4 Books Collectio...,Harry Potter and the Order of the Phoenix (Boo...,158
2031192,Harry Potter and the Sorcerer's Stone,Harry Potter and the Order of the Phoenix (Boo...,146


##### Frequently read with Harry Potter And The Goblet Of Fire

Top 10 books frequently read with `Harry Potter And The Goblet Of Fire`

In [51]:
harry_potter = true_pairs[true_pairs["book_a"] == "Harry Potter And The Goblet Of Fire"].nlargest(10, "size")

In [52]:
harry_potter

,book_a,book_b,size
2010197,Harry Potter And The Goblet Of Fire,Harry Potter and the Order of the Phoenix (Boo...,175
2010195,Harry Potter And The Goblet Of Fire,Harry Potter The Illustrated 4 Books Collectio...,129
2010199,Harry Potter And The Goblet Of Fire,Harry Potter and the Sorcerer's Stone,113
2010192,Harry Potter And The Goblet Of Fire,Harry Potter And The Prisoner Of Azkaban,83
2010200,Harry Potter And The Goblet Of Fire,Harry Potter and the Sorcerer's Stone (1),61
2010196,Harry Potter And The Goblet Of Fire,Harry Potter and the Chamber of Secrets,51
2011769,Harry Potter And The Goblet Of Fire,The Fellowship of the Ring (The Lord of the Ri...,33
2011779,Harry Potter And The Goblet Of Fire,The Firm,24
2011650,Harry Potter And The Goblet Of Fire,The Client,21
2012624,Harry Potter And The Goblet Of Fire,Where the Heart Is,21


Most people who read `Harry Potter and The Goblet Of Fire` also read `Harry Potter and the Order of the Phoenix`

#### Serialize `userId` and `bookId` columns. 

In order to avoid problems with PySpark mishandling column info, the two feature columns should be manually set to integers in the range `0...N-1`. When this is not the case, PySpark will sometimes generate arbitrary `bookId` and `userId` numbers. While these automatically generated numbers can be used well enough within the ALS model, they make it impossible to re-map recommendations back to the original books dataframe.

In [53]:
books_df.user_id = pd.Categorical(books_df.user_id)
books_df["userId"] = books_df.user_id.cat.codes

In [54]:
books_df.isbn13 = pd.Categorical(books_df.isbn13)
books_df["bookId"] = books_df.isbn13.cat.codes

In [55]:
books_df = books_df.rename(columns={"book_rating": "rating"})

In [56]:
clean_df = books_df[["userId", "bookId", "title", "language", "rating"]]

### Save to Parquet

Save file to parquet. This makes ingesting the data into PySpark easier. 

In [57]:
clean_df.to_parquet('working_frame.parquet', index=False)

In [58]:
client.close()

# Spark

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType

In [60]:
spark = SparkSession.builder \
        .appName("Group7") \
        .config("spark.sql.repl.eagerEval.enabled", True) \
        .config("spark.sql.repl.eagerEval.maxNumRows", 10) \
        .config("spark.driver.memory", "4g") \
        .getOrCreate()

23/04/14 16:34:46 WARN Utils: Your hostname, kobuta resolves to a loopback address: 127.0.1.1; using 172.28.223.61 instead (on interface eth0)
23/04/14 16:34:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/14 16:34:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [61]:
spark

In [62]:
spark_df = spark.read.parquet("working_frame.parquet")

In [63]:
spark_df.printSchema()

root
 |-- userId: short (nullable = true)
 |-- bookId: short (nullable = true)
 |-- title: string (nullable = true)
 |-- language: string (nullable = true)
 |-- rating: long (nullable = true)



The columns `userId`, `bookId` and `rating` should be of type `int` for ALS to work.

In [64]:
spark_df = spark_df.withColumn("userId", F.col("userId").cast(IntegerType())) \
                .withColumn("bookId", F.col("bookId").cast(IntegerType())) \
                .withColumn("rating", F.col("rating").cast(IntegerType())) 


In [65]:
spark_df

userId,bookId,title,language,rating
179,2872,A Painted House: ...,en,10
277,2872,A Painted House: ...,en,9
358,2872,A Painted House: ...,en,7
382,2872,A Painted House: ...,en,10
585,2872,A Painted House: ...,en,6
611,2872,A Painted House: ...,en,8
638,2872,A Painted House: ...,en,8
648,2872,A Painted House: ...,en,5
1009,2872,A Painted House: ...,en,8
1024,2872,A Painted House: ...,en,8


In [66]:
book_ratings = spark_df.select("userId", "bookId", "rating")

In [67]:
book_ratings

userId,bookId,rating
179,2872,10
277,2872,9
358,2872,7
382,2872,10
585,2872,6
611,2872,8
638,2872,8
648,2872,5
1009,2872,8
1024,2872,8


### Sparcity

In [68]:
n_ratings = book_ratings.count()
n_ratings

98451

In [69]:
n_users = book_ratings.select("userId").distinct().count()
n_users

5333

In [70]:
n_books = book_ratings.select("bookId").distinct().count()
n_books

9259

In [71]:
sparsity = 1.0 - (n_ratings/(n_users * n_books))
sparsity

0.9980061868133958

In [72]:
book_ratings.groupBy("userId").count()

userId,count
4900,12
496,8
1580,20
2659,15
3918,33
5300,7
1829,7
5156,44
833,9
4818,6


In [73]:
book_ratings.filter(F.col("userId") == 4818)

userId,bookId,rating
4818,6392,7
4818,895,5
4818,363,5
4818,1183,7
4818,1183,7
4818,7027,8


## Spark ML

In [74]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit

#### Train-Test Split

In [75]:
(train, test) = book_ratings.randomSplit([0.8, 0.2], seed=42)

In [76]:
train.cache()
test.cache();

### Simple ALS Model

First generate a simple ALS model with default hyperparameters, then fit the model to the training data and predict the ratings a user would give a book. 

In [77]:
als = ALS(
    userCol="userId", 
    itemCol="bookId", 
    ratingCol="rating", 
    nonnegative=True, 
    coldStartStrategy="drop", 
    seed=42)

In [78]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [79]:
simple_model = als.fit(train)

In [80]:
predictions = simple_model.transform(test)

In [81]:
predictions

userId,bookId,rating,prediction
148,3473,10,10.002486
148,5909,7,8.571876
463,357,8,4.871826
463,867,5,8.357307
471,1290,10,8.063271
471,6153,10,6.145279
496,3655,9,9.084566
833,915,5,6.288861
833,2056,5,4.5948772
1088,3760,7,7.8322515


In [82]:
predictions.filter(F.col("prediction") > 10.0).count()

468

In [83]:
rmse = evaluator.evaluate(predictions)
rmse

1.8919501431351162

### Hyperparameter Tuning

The simple model shows very lucklustre performance with `rmse = 1.89`, i.e. every prediction is off by around ±2.  

Create another model with hyperparameter tuning. 

**Caution:** Since Spark calculations are computationally expensive, utmost care should be taken when running on personal machines or weak clusters. Reduce the number of hyperparameter combinations to avoid system crashes

In [84]:
from tuning import RandomGridBuilder

In [85]:
param_grid = RandomGridBuilder(4)\
            .addDistr(als.rank, lambda :  np.random.randint(50, 150)) \
            .addDistr(als.maxIter, lambda : np.random.randint(10, 20)) \
            .addDistr(als.regParam, lambda : np.random.uniform(0.1, 0.15)) \
            .build()

In [86]:
# cv = CrossValidator(
#     estimator=als, 
#     estimatorParamMaps=param_grid, 
#     evaluator=evaluator,
#     parallelism=4,
#     numFolds=5)

In [87]:
cv = TrainValidationSplit(
    estimator=als, 
    estimatorParamMaps=param_grid, 
    evaluator=evaluator, 
    parallelism=4, 
    trainRatio=0.80)

In [88]:
print(f"Num. of models to run: {len(param_grid)}")

Num. of models to run: 4


In [89]:
tuned_model = cv.fit(train)

In [90]:
best_model = tuned_model.bestModel

In [91]:
predictions_2 = best_model.transform(test)

In [92]:
predictions_2

userId,bookId,rating,prediction
1395,6654,10,6.6222067
2563,463,7,6.9356494
2563,3749,8,6.429162
1884,8389,10,7.3435364
5140,7982,6,6.6458287
847,4519,7,8.903383
5055,2366,8,6.4989185
2463,2366,10,8.87252
4443,5300,7,6.543282
4230,8638,10,9.18901


In [93]:
rmse_2 = evaluator.evaluate(predictions_2)
rmse_2

1.7356805639598107

A marginal improvement, but still disappointing

---
## Recommendations

### Part I. Top N for all users

Generate top 5 book recommendations for **all** users.

In [94]:
user_recs = best_model.recommendForAllUsers(5)

In [95]:
user_recs

userId,recommendations
26,"[{9032, 9.9216175..."
27,"[{7045, 10.809097..."
28,"[{7045, 10.115082..."
31,"[{9032, 10.432951..."
34,"[{7045, 10.607924..."
44,"[{6533, 9.60229},..."
53,"[{9032, 10.545756..."
65,"[{9032, 9.808269}..."
76,"[{7045, 9.852566}..."
78,"[{3324, 10.074556..."


#### Re-write from Vector to Tabular form

The output is hard not very user-friendly. Transform the users recommendations table `user_recs` from Wide to Long Format. Each of the five book recommendations can then be represented in individual rows.

Create a temporary view from the `user_recs` table, and run the sql query below. The resulting table is a preview showing book an exploded view of the book recommendations for the first two users (`userId=26` and `userId=27`). The table also shows what the model predicts the each user will rate a given book.

Because of the nature of regression being on a continuous scale, some predicted values are greater than 10. In such cases the model simply predicts that a reader/user would give a particular book a very "high" rating.

In [96]:
user_recs.createTempView("ALS_recs_temp")

In [97]:
query="""
SELECT 
    userId AS targetId,
    bookIds_and_ratings.bookId,
    bookIds_and_ratings.rating AS prediction
FROM ALS_recs_temp
LATERAL VIEW explode(recommendations) exploded_table
AS bookIds_and_ratings
"""

clean_recs = spark.sql(query)
clean_recs

targetId,bookId,prediction
26,9032,9.9216175
26,2171,9.9216175
26,8930,9.9216175
26,8937,9.9216175
26,8185,9.486184
27,7045,10.809097
27,9032,10.323879
27,8930,10.323879
27,2171,10.323879
27,8937,10.323879


#### Zoom in

Examine recommendations for the first reader in the dataframe, i.e.`targetId` = 26. To accomplish this, the recommendations dataframe `clean_recs` is left-merged using `bookId` into the original spark dataframe contain that contains full info about a book.

This is then filtered to find book recommendations for the reader with `targetId=26` as shown below:

In [98]:
merged_data = clean_recs.join(spark_df, on=["bookId"], how="left").orderBy("bookId").drop_duplicates(['bookId'])

In [99]:
user26 = merged_data.where(F.col("targetId") == 26)
user26

bookId,targetId,prediction,userId,title,language,rating
2171,26,9.9216175,3428,Dark Lady,en,10
8185,26,9.486184,4529,Frankenstein,en,10
8930,26,9.9216175,3428,Wish You Well,en,10
8937,26,9.9216175,3428,And Then There We...,en,10
9032,26,9.9216175,3428,A Bend in the Road,en,10


The table above clearly shows five books recommended to the user with `targetId =26`. It also shows the targert user's predicted ratings for each book, as well as actual ratings from similar users as determined by the model

---
### Part II. Top N for all users

Generate top 5 user recommendations for each book, i.e. given a certain book, find 5 users who might be interested. The result is given below.

In [100]:
book_recs = best_model.recommendForAllItems(5)

In [101]:
book_recs

bookId,recommendations
28,"[{3315, 10.210461..."
31,"[{2022, 11.01064}..."
53,"[{2165, 9.711294}..."
65,"[{5306, 8.5152645..."
76,"[{3490, 9.612458}..."
78,"[{3850, 9.398474}..."
81,"[{3028, 6.8598533..."
85,"[{1222, 7.445954}..."
101,"[{869, 10.861304}..."
108,"[{2022, 6.118682}..."


In [102]:
book_recs.createOrReplaceTempView("ALS_books_temp")

In [103]:
query="""
SELECT 
    bookId,
    bookIds_and_userIds.userId AS targetId,
    bookIds_and_userIds.rating AS predicted_rating
FROM ALS_books_temp
LATERAL VIEW explode(recommendations) exploded_table
AS bookIds_and_userIds
"""
clean_book_recs = spark.sql(query)

---
### Detailed Information


In [104]:
expr = [F.avg('rating'), F.count('rating')]

spark_df.where("bookId == 28").groupby(["bookId", "title"]).agg(*expr)

bookId,title,avg(rating),count(rating)
28,"Over Sea, Under S...",8.333333333333334,9


----
The book `Over Sea, Under Stone` has 9 ratings and an average user rating of 8.3. The model recommends it to the following five users:

In [105]:
clean_book_recs.where(F.col("bookId") == 28)

bookId,targetId,predicted_rating
28,3315,10.210461
28,67,9.916276
28,869,9.879538
28,2485,9.793296
28,3604,9.723386


#### Shutdown Spark

In [104]:
spark.stop()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1203c34a-9648-4e70-9ae7-f7a7928e842f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>